In [ ]:
%pip install accelerate -U -qqq
%pip install transformers[torch] -qqq

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import yaml
from distutils.dir_util import copy_tree

In [3]:
import torch
import yaml
from src.data.datamodule import DataManager

In [6]:
import json
import os
import numpy as np
import re

from tqdm import tqdm

from src.data.mt_dataset import MTDataset_HF
from src.data.tokenizers.unif_tokenizers import UNIFTokenizer

In [60]:
data_config = yaml.load(open("configs/data_config.yaml", 'r', encoding='utf-8'), Loader=yaml.Loader)
data_path = lambda x: data_config["path_repository"] + "data/" + data_config["data_language"] + str(x) + data_config["data_name_file"] + ".json"

In [61]:
config = data_config
device = "cpu"

tokenizer = UNIFTokenizer(path_tok=config["path_repository"] + "data/query_vocab.json",
                               pre_train_name=config["pre_train_tokenizer"],
                               pad_flag=True,
                               max_length=config["max_sent_len"])

def prepare_data(path_data, drop_last=False):

    dev_data = json.load(open(os.path.join(path_data), 'r', encoding="utf-8"))
    target_sentences = []
    source_sentences = []
    for sample in tqdm(dev_data[:config["separate_batch"]], desc="Pars data"):
        target_sentences.append(sample['masked_query'])
        source_sentences.append(sample['question'])

    # DataLoader

    tokenized_source_sentences = [tokenizer.tkr(i) for i in source_sentences]#[0:10]
    tokenized_target_sentences = [tokenizer.tkr(i) for i in target_sentences]#[0:10]

    dataset = MTDataset_HF(tokenized_source_list=tokenized_source_sentences,
                        tokenized_target_list=tokenized_target_sentences, device=device)
    return dataset

In [62]:
dev_dataloader = prepare_data(path_data=data_path("dev"), drop_last=False)
test_dataloader = prepare_data(path_data=data_path("test"), drop_last=True)


Pars data: 100%|██████████| 15877/15877 [00:00<00:00, 1578705.72it/s]


In [63]:
from transformers import XLMRobertaForCausalLM, AutoConfig
config = AutoConfig.from_pretrained("FacebookAI/roberta-base")
config.is_decoder = True
model = XLMRobertaForCausalLM.from_pretrained("FacebookAI/roberta-base", config=config).to('cuda')

In [64]:
sum(p.numel() for p in model.parameters())

124697433

In [65]:
from transformers import Trainer, TrainingArguments

In [66]:
training_args = TrainingArguments(
    f"roberta-base-exp",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,# 2e-5,
    weight_decay=0.01,
    num_train_epochs = 3,
    logging_dir = 'logs',
    save_strategy="no"
)

In [67]:
from transformers import ProgressCallback, PrinterCallback

In [68]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dev_dataloader,
    eval_dataset=test_dataloader,
    callbacks = [PrinterCallback]
)

In [69]:
end_train = trainer.train()

 16%|█▌        | 500/3159 [00:30<02:40, 16.53it/s]

{'loss': 1.1346, 'learning_rate': 1.6834441278885725e-05, 'epoch': 0.47}
{'loss': 1.1346, 'learning_rate': 1.6834441278885725e-05, 'epoch': 0.47}


 32%|███▏      | 1000/3159 [01:01<02:12, 16.31it/s]

{'loss': 0.8333, 'learning_rate': 1.3668882557771448e-05, 'epoch': 0.95}
{'loss': 0.8333, 'learning_rate': 1.3668882557771448e-05, 'epoch': 0.95}


100%|█████████▉| 1981/1985 [00:33<00:00, 58.99it/s]
                                                   
 33%|███▎      | 1054/3159 [01:38<3:31:03,  6.02s/it]A

...

{'loss': 0.7033, 'learning_rate': 4.172206394428617e-06, 'epoch': 2.37}
{'loss': 0.7033, 'learning_rate': 4.172206394428617e-06, 'epoch': 2.37}


 95%|█████████▌| 3003/3159 [04:10<00:09, 16.31it/s]

{'loss': 0.6926, 'learning_rate': 1.00664767331434e-06, 'epoch': 2.85}
{'loss': 0.6926, 'learning_rate': 1.00664767331434e-06, 'epoch': 2.85}


100%|█████████▉| 1984/1985 [00:33<00:00, 58.92it/s]
                                                   
100%|██████████| 3159/3159 [04:54<00:00, 10.74it/s]

{'eval_loss': 0.7546016573905945, 'eval_runtime': 33.8176, 'eval_samples_per_second': 469.489, 'eval_steps_per_second': 58.697, 'epoch': 3.0}
{'eval_loss': 0.7546016573905945, 'eval_runtime': 33.8176, 'eval_samples_per_second': 469.489, 'eval_steps_per_second': 58.697, 'epoch': 3.0}
{'train_runtime': 294.1969, 'train_samples_per_second': 85.861, 'train_steps_per_second': 10.738, 'train_loss': 0.802581847487616, 'epoch': 3.0}
{'train_runtime': 294.1969, 'train_samples_per_second': 85.861, 'train_steps_per_second': 10.738, 'train_loss': 0.802581847487616, 'epoch': 3.0}


In [70]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

100%|██████████| 1985/1985 [00:33<00:00, 59.24it/s]

{'eval_loss': 0.7546016573905945, 'eval_runtime': 33.5232, 'eval_samples_per_second': 473.612, 'eval_steps_per_second': 59.213, 'epoch': 3.0}
Perplexity: 2.13


In [87]:
import random

In [88]:
trainer.model.to('cpu');
for _ in range(5):
    index = random.randint(0,len(test_dataloader)-1)
    print(f"---------true-----------")
    print(tokenizer.decode(test_dataloader[index]['input_ids']), tokenizer.decode(test_dataloader[index]['labels']))
    print(f"---------predict-----------")
    predict = trainer.model.generate(test_dataloader[index]['input_ids'].reshape((1,-1)), max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)[0]
    print(tokenizer.decode(predict))

---------true-----------
[CLS] What is the number in season of the episode whose production code is pabf05 ? [SEP]                    [CLS] SELECT # FROM table WHERE Production_code = STR_VALUE_1 [SEP]                              
---------predict-----------
[CLS] What is the number in season of the episode whose production code is pabf05 ? [SEP]                                SELECT   SELECT        SELECT  SELECT     SELECT  SELECT   SELECT  SELECT  
---------true-----------
[CLS] What is the highest pick number for player don barber ? [SEP]                           [CLS] SELECT MAX Pick_# FROM table WHERE Player = STR_VALUE_1 [SEP]                             
---------predict-----------
[CLS] What is the highest pick number for player don barber ? [SEP]                           SELECT Venue WHERE  SELECT Home_team   SELECT AND    SELECT         SELECT         SELECT FROM FROM Position Score Title = NUM_VALUE_1 Player
---------true-----------
[CLS] What was the original air date f

In [90]:
trainer.model.to('cpu');
for _ in range(4):
    index = random.randint(0,len(dev_dataloader)-1)
    print(f"---------true-----------")
    print(tokenizer.decode(dev_dataloader[index]['input_ids']), tokenizer.decode(dev_dataloader[index]['labels']))
    print(f"---------predict-----------")
    predict = trainer.model.generate(dev_dataloader[index]['input_ids'].reshape((1,-1)), max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)[0]
    print(tokenizer.decode(predict))

---------true-----------
[CLS] What is the Deputy's affiliation in 1992 – 93 ? [SEP]                           [CLS] SELECT Deputy's_affiliation FROM table WHERE Year = STR_VALUE_1 [SEP]                              
---------predict-----------
[CLS] What is the Deputy's affiliation in 1992 – 93 ? [SEP]                           SELECT Result  SELECT     SELECT Grid Grid      SELECT FROM       SELECT  SELECT   SELECT FROM FROM table FROM Rank = = = NUM_VALUE_1 Score
---------true-----------
[CLS] What was the first leg of the semi - final ? [SEP]                            [CLS] SELECT First_leg FROM table WHERE Round = STR_VALUE_1 [SEP]                              
---------predict-----------
[CLS] What was the first leg of the semi - final ? [SEP]                            SELECT Bronze FROM  SELECT   SELECT STR_VALUE_3 AND Total = Opponent WHERE = Points = = = =                    
---------true-----------
[CLS] Which venue had an extra of Junior Race ? [SEP]                      

In [76]:
metrics = end_train.metrics

# save train results
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

***** train metrics *****
  epoch                    =        3.0
  train_loss               =     0.8026
  train_runtime            = 0:04:54.19
  train_samples_per_second =     85.861
  train_steps_per_second   =     10.738
